In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
import requests


def download(url: str, dest_folder: str):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist

    filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
    file_path = os.path.join(dest_folder, filename)

    r = requests.get(url, stream=True)
    if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))


In [3]:
data = pd.read_csv("/home/nguu0123/Downloads/images_2016_08_v5/images_2016_08/validation/images.csv")
data['ImageID'] = data['ImageID'].astype(str)
data.head()

,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL
0,000026e7ee790996,validation,https://farm7.staticflickr.com/5769/2109480371...,https://www.flickr.com/photos/132646954@N02/21...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/132646954@N02/,dronepicr,Chichen Itza Mexico,7825604,EEGQ0ZQ/OxJAIGzfJ6LMvw==,https://c1.staticflickr.com/6/5769/21094803716...
1,000062a39995e348,validation,https://c1.staticflickr.com/4/3743/11539980784...,https://www.flickr.com/photos/deniwlp84/115399...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/deniwlp84/,Deni Williams,Parque Zoológico de São Paulo / Sao Paulo Zoo,4227077,oc+H0KgW0sxyEgAlfOiQDQ==,https://c3.staticflickr.com/4/3743/11539980784...
2,0000c64e1253d68f,validation,https://c8.staticflickr.com/7/6007/6010263871_...,https://www.flickr.com/photos/ladydragonflyher...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/ladydragonflyher...,LadyDragonflyCC - &gt;;&lt;,Redneck Figure 8's,1744183,Xufa/rhlOnBQSotR/8Gqhg==,https://c7.staticflickr.com/7/6007/6010263871_...
3,000132c20b84269b,validation,https://c4.staticflickr.com/9/8555/15625756039...,https://www.flickr.com/photos/ministeriodesalu...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/ministeriodesalu...,Ministerio de Salud de Neuquen,consultorio ecografia piedra del Aguila (4),461828,uPw/ycB1skL/byjF0AYJKA==,https://c3.staticflickr.com/9/8555/15625756039...
4,0001eeaf4aed83f9,validation,https://c2.staticflickr.com/6/5606/15611395595...,https://www.flickr.com/photos/hisgett/15611395595,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/hisgett/,Tony Hisgett,American Airlines Boeing 777-323(ER) N717AN,2038323,I4V4qq54NBEFDwBqPYCkDA==,https://c4.staticflickr.com/6/5606/15611395595...


In [4]:
data["Thumbnail300KURL"][2]

'https://c7.staticflickr.com/7/6007/6010263871_df93fb6382_z.jpg'

In [5]:
label = pd.read_csv("/home/nguu0123/Downloads/human_ann_2016_08_v3/human_ann_2016_08/validation/labels.csv")
label_name = pd.read_csv("/home/nguu0123/Downloads/human_ann_2016_08_v3/human_ann_2016_08/validation/dict.csv", names=['LabelName','Label'])
label = label.merge(label_name, on='LabelName', how='inner')
label.head()

,ImageID,Source,LabelName,Confidence,Label
0,000026e7ee790996,human,/m/01cbzq,1.0,rock
1,0047528a9aac4cc5,human,/m/01cbzq,1.0,rock
2,0053092bf2b1d957,human,/m/01cbzq,1.0,rock
3,00627adda9a83f9d,human,/m/01cbzq,1.0,rock
4,00703ba8b466d2ad,human,/m/01cbzq,1.0,rock


In [6]:
label['ImageID'] = label['ImageID'].astype(str)

In [7]:
data_with_label = data.merge(label, on='ImageID', how='inner')
data_with_label = data_with_label[['ImageID','Thumbnail300KURL','Source','Label','Confidence']]

In [8]:
print(data_with_label.size)
data_with_label = data_with_label[data_with_label['Thumbnail300KURL'].notnull()]
print(data_with_label.size)

8706920
8426810


In [33]:
data = data_with_label[data_with_label['Label'] == 'cat']['Thumbnail300KURL'].iloc[5]
data[data.rfind('/') + 1: data.rfind('.')]


'258460789_c4baeb300a_z'

In [ ]:
# yolo_class = ['cat','dog','vehicle','bicycle','person']
# for cur_class in yolo_class:
#     data = data_with_label[data_with_label['Label'] == cur_class]
#     for i in range(100):
#         download(data['Thumbnail300KURL'].iloc[i], cur_class) 

In [47]:
# path = os.getcwd()
# for cur_class in yolo_class:
#     cur_path = path + '/' + cur_class
#     files = os.listdir(cur_path)
#     os.chdir(cur_path)
#     for file in files:
#         if('?' in file):
#             os.rename(file, file[:file.index('?')])

In [1]:
from yolov8.yolov8 import Yolo8
from yolov5.yolov5 import Yolo5
import numpy as np
import os
yolov5 = Yolo5("yolov5s")
yolov8 = Yolo8("yolov8n")

YOLOv5 🚀 v7.0-72-g064365d Python-3.9.15 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070, 7960MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [2]:
import cv2
def enhance_image(image):
    kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
    enhanced_im = cv2.filter2D(src=image, ddepth=-1, kernel=kernel)
    return enhanced_im
def get_prediction(list_pre, i):
    pre_item = list_pre[i]
    keys = list(pre_item.keys())
    return pre_item[keys[0]]
def extract_dict(dict, keys):
    result = {}
    for key in keys:
        result[key] = dict[key]
    return result
def not_approximate(a,b):
    if abs(a-b)< 10:
        return False
    else: 
        return True

def compare_box(box1, box2):
    for key in box1:
        if not_approximate(box1[key], box2[key]):
            return False
    return True

def agg_mean(predict_list):
    merged_dict = predict_list[0]
    for i in range(1,len(predict_list)):
        for key in predict_list[i]:
            if key not in merged_dict:
                merged_dict[key] = predict_list[i][key]
            else:
                merged_dict[key] += predict_list[i][key]
    for key in merged_dict:
        merged_dict[key] /=  len(predict_list)
    return merged_dict

def agg_max(predictions):
    pre_list = []
    agg_prediction = []
    object_count = 0
    for key in predictions:
        pre_list += predictions[key]
    while pre_list:
        pre_item = get_prediction(pre_list,0)
        box1 = extract_dict(pre_item[0],["xmin", "ymin", "xmax", "ymax"])
        duplicate = []
        for i in range(1,len(pre_list)):
            box2 = extract_dict(get_prediction(pre_list,i)[0],["xmin", "ymin", "xmax", "ymax"])
            if compare_box(box1,box2):
                pre_item += get_prediction(pre_list,i)
                duplicate.append(i)
        for item in duplicate:
            pre_list.pop(item)
        max_item = pre_item[0]
        for item in pre_item:
            if item["confidence"] > max_item["confidence"]:
                max_item = item
        detect_obj = {f"object_{object_count}":max_item}
        pre_list.pop(0)
        object_count += 1
        agg_prediction.append(detect_obj)
    return agg_prediction
    

In [4]:
object_class = ['cat','dog','car','bicycle','person']
for cur_class in object_class:
     path = os.getcwd() + '/images/' + cur_class
     files = os.listdir(path)
     yolo5_confidence = []
     yolo8_confidence = []
     yolo5_true  = 0
     yolo5_false = 0
     yolo8_true  = 0
     yolo8_false = 0
     i = 0
     for file in files:
         image = cv2.imread(path + '/' + file)
         en_im = enhance_image(image)
         result_yolo5 = yolov5.yolov5_inference(en_im)
         if result_yolo5 is not None:
             for cur_object in result_yolo5[0]['yolov5s'][0]:
                 data = result_yolo5[0]['yolov5s'][0][cur_object][0]
                 if data['name'] == cur_class:
                     yolo5_true +=1
                 else:
                     yolo5_false += 1
                 yolo5_confidence.append(data['confidence'])
         result_yolo8 = yolov8.yolov8_inference(en_im)
         if result_yolo8[0]['yolov8n'] != []:
             for cur_object in result_yolo8[0]['yolov8n'][0]:
                 data = result_yolo8[0]['yolov8n'][0][cur_object][0]
                 if data['name'] == cur_class:
                     yolo8_true +=1
                 else:
                     yolo8_false += 1
                 yolo8_confidence.append(data['confidence'])
     print(cur_class)
     print('yolov5 confidence {} true detect {} false detect {}'.format(np.average(yolo5_confidence), yolo5_true, yolo5_false))
     print('yolov8 confidence {} true detect {} false detect {}'.format(np.average(yolo8_confidence), yolo8_true, yolo8_false))


Ultralytics YOLOv8.0.25 🚀 Python-3.9.15 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070, 7960MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


cat
yolov5 confidence 0.5109227895736694 true detect 60 false detect 15
yolov8 confidence 0.7363022177718407 true detect 77 false detect 9
dog
yolov5 confidence 0.5719827995242843 true detect 48 false detect 35
yolov8 confidence 0.6582733710388561 true detect 49 false detect 42
car
yolov5 confidence 0.6790541684941241 true detect 38 false detect 38
yolov8 confidence 0.697952942877281 true detect 35 false detect 47
bicycle
yolov5 confidence 0.6327341339167427 true detect 30 false detect 55
yolov8 confidence 0.7514804361096348 true detect 39 false detect 44
person
yolov5 confidence 0.7052051137233603 true detect 81 false detect 6
yolov8 confidence 0.8146213729950514 true detect 83 false detect 5
